In [1]:
type SignalType = 
    | Buy
    | Sell
    | SignalUndefined

type PivotType = 
    | High
    | Low
    | PivotUndefined

type StatCandle = {
    Open: decimal
    High: decimal
    Low: decimal
    Close: decimal
    Time: string
    Volume: decimal
    ChannelLow: decimal
    ChannelHigh: decimal
    PivotType: PivotType
    PivotPrice: decimal
    BreakoutSignal: SignalType
    TradeCapital: decimal
}

In [9]:
#r "nuget: Dapper, 2.1.28"
#r "nuget: MathNet.Numerics, 5.0.0"
#r "nuget: System.Data.SQLite, 1.0.118"
open System.Data.SQLite
open Dapper

let loadData () =
    let databaseFilename = "analytics.sqlite"
    let connectionStringFile = sprintf "Data Source=%s;Version=3;" databaseFilename  

    use connection = new SQLiteConnection(connectionStringFile)
    connection.Open()

    let sql = "SELECT * FROM symbols"


Installed Packages Dapper, 2.1.28 MathNet.Numerics, 5.0.0 System.Data.SQLite, 1.0.118

Error: input.fsx (11,1)-(11,4) parse error This is not a valid identifier

In [3]:
#r "nuget: XPlot.Plotly, 4.0.6"

open XPlot.Plotly
open System

let projectPath = "c:/users/tyueksel/desktop/f#/reaper_f/src/analytics"

let targetFile = projectPath + "/patterns/price-data/broadening_bottoms.csv"

let candles = parseCsv targetFile


Installed Packages XPlot.Plotly, 4.0.6

In [4]:
let pivotScatter(candles: StatCandle array) (pivotType: PivotType) : Trace = 
    let pivots = 
        candles
        |> Array.filter (fun candle -> candle.PivotType = pivotType)

    let times = 
        pivots
        |> Array.map (fun candle -> candle.Time)

    let prices = 
        pivots
        |> Array.map (fun candle -> candle.PivotPrice)

    Scatter(x = times, y = prices, mode = "markers", name = $"Pivot {pivotType}")

    

In [5]:
let candleStickTrace (candles: StatCandle array) : Trace =
    Candlestick( 
        x = (candles |> Array.map (fun r -> r.Time)), 
        ``open`` = (candles |> Array.map (fun r -> r.Open)), 
        high = (candles |> Array.map (fun r -> r.High)), 
        low = (candles |> Array.map (fun r -> r.Low)), 
        close = (candles |> Array.map (fun r -> r.Close)), 
        name = "Candlestick")



In [6]:
let channelLowTrace (candles: StatCandle array) : Trace =
    let channelLow = Scatter(x = (candles |> Array.map (fun r -> r.Time)),
        y = (candles |> Array.map (fun r -> r.ChannelLow)),
        mode = "lines",
        name = "Channel Low")
    channelLow


let channelHighTrace (candles: StatCandle array) : Trace =
    let channelHigh = Scatter(x = (candles |> Array.map (fun r -> r.Time)),
        y = (candles |> Array.map (fun r -> r.ChannelHigh)),
        mode = "lines",
        name = "Channel High")
    channelHigh

In [7]:
let breakoutActionTrace (candles: StatCandle array) : Trace =
    let breakoutActions = 
        candles
        |> Array.filter (fun candle -> candle.BreakoutSignal <> SignalUndefined)

    let times = 
        breakoutActions
        |> Array.map (fun candle -> candle.Time)

    let prices = 
        breakoutActions
        |> Array.map (fun candle -> candle.Close)

    Scatter(x = times, y = prices, mode = "markers", name = "Breakout Actions")

In [8]:
let candleStickTrace = candleStickTrace candles
let breakoutActionTrace = breakoutActionTrace candles
let pivotHighScatters = pivotScatter candles High
let pivotLowScatters = pivotScatter candles Low


let candleChunks = 
    candles
    |> Array.splitInto 3


let channelLowTraces = 
    candleChunks
    |> Array.map (fun chunk -> channelLowTrace chunk)

let channelHighTraces = 
    candleChunks
    |> Array.map (fun chunk -> channelHighTrace chunk)

let chart = Chart.Plot(
    Array.concat [| 
        [|
            candleStickTrace
            breakoutActionTrace
            pivotHighScatters
            pivotLowScatters 
        |]
        channelLowTraces
        channelHighTraces
    |])

chart.WithLayout(Layout(title = "Broadening Bottoms",
    xaxis = Xaxis(title = "Time"),
    yaxis = Yaxis(title = "Price")))

chart.Show()
